<div style="text-align: center">
    <h3>Essay Score Estimator</h3>
</div>
<p style="text-align: center">
    How Jia Jean<br>
    <i>30th April 2021</i>
</p>   __________________________________________________________________________________________________________________________________

# Introduction

The objective of this study is to perform predictive analysis throught supervised machine learning on Essay-Features.csv dataset. The dataset contains the students' essay scoring and features that might have correspondent effect on the essay scoring such as characters, words, punctuations, average word length, synonym words, and others. The dataset undergone the process below:

1. Import Libraries
2. Importing, description, and modification of Data
3. Supervised Learning
    - Normalisation
    - Classification
4. Support Vector Machine (SVM)
    - Quadratic Weighted Kappa (QWK)
5. Kaggle Dataset 
6. Conclusion
7. References


## 1. Import Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

## 2. Importing, Description, and Modification of Data

In [4]:
df1 = pd.read_csv('assignment2_data/Essay-Features.csv')

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   essayid                    1332 non-null   int64  
 1   chars                      1332 non-null   int64  
 2   words                      1332 non-null   int64  
 3   commas                     1332 non-null   int64  
 4   apostrophes                1332 non-null   int64  
 5   punctuations               1332 non-null   int64  
 6   avg_word_length            1332 non-null   float64
 7   sentences                  1332 non-null   int64  
 8   questions                  1332 non-null   int64  
 9   avg_word_sentence          1332 non-null   float64
 10  POS                        1332 non-null   float64
 11  POS/total_words            1332 non-null   float64
 12  prompt_words               1332 non-null   int64  
 13  prompt_words/total_words   1332 non-null   float

In [6]:
df1.describe()

,essayid,chars,words,commas,apostrophes,punctuations,avg_word_length,sentences,questions,avg_word_sentence,POS,POS/total_words,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score
count,1332.00000,1332.000000,1332.000000,1332.000000,1332.000000,1332.00000,1332.000000,1332.000000,1332.000000,1332.000000,1332.000000,1332.000000,1332.000000,1332.000000,1332.00000,1332.000000,1332.000000,1332.000000,1332.000000
mean,905.27027,2101.745495,424.485736,14.667417,8.141141,0.47973,4.939762,19.704204,1.222973,23.884687,420.596542,0.989935,198.913664,0.469164,110.16967,0.263846,468.987988,455.507508,3.427177
std,526.68760,865.963750,171.873730,10.920781,6.124520,1.27168,0.231071,19.202731,1.847446,11.160020,170.985111,0.007308,82.729266,0.052466,43.96192,0.038870,159.447449,155.751220,0.774275
min,0.00000,169.000000,36.000000,0.000000,2.000000,0.00000,2.231322,0.000000,0.000000,1.084112,35.647059,0.924771,14.000000,0.288889,11.00000,0.027299,48.000000,50.000000,1.000000
25%,442.75000,1527.250000,310.000000,7.000000,4.000000,0.00000,4.791679,13.000000,0.000000,19.142857,305.406284,0.987758,144.000000,0.435709,81.00000,0.238423,361.000000,350.750000,3.000000
50%,914.50000,2029.500000,411.000000,13.000000,6.000000,0.00000,4.946059,18.000000,1.000000,22.030331,406.982869,0.991572,193.000000,0.465852,107.50000,0.262872,463.000000,448.000000,3.000000
75%,1369.75000,2613.500000,525.000000,21.000000,11.000000,0.00000,5.092938,24.000000,2.000000,26.048234,520.739458,0.994425,246.000000,0.500000,134.00000,0.288277,581.000000,561.250000,4.000000
max,1799.00000,6142.000000,1170.000000,72.000000,51.000000,26.00000,5.681429,642.000000,17.000000,303.000000,1158.984563,1.000000,669.000000,0.961207,355.00000,0.465517,750.000000,750.000000,6.000000


### Modifications in df1

In [7]:
df1['stemmed/total_words'] = df1['stemmed']/df1['words']
df1['unstemmed/total_words'] = df1['unstemmed']/df1['words']
df1['related_words'] = df1['prompt_words'] + df1['POS'] + df1['synonym_words']

In [8]:
df1.sample(5)

,essayid,chars,words,commas,apostrophes,punctuations,avg_word_length,sentences,questions,avg_word_sentence,...,prompt_words,prompt_words/total_words,synonym_words,synonym_words/total_words,unstemmed,stemmed,score,stemmed/total_words,unstemmed/total_words,related_words
270,805,2422,495,9,13,0,4.892929,15,6,33.000000,...,230,0.464646,130,0.262626,602,585,4,1.181818,1.216162,850.653117
767,1343,2581,510,22,20,0,5.060784,24,0,21.250000,...,255,0.500000,119,0.233333,456,448,4,0.878431,0.894118,880.318898
1200,1597,2223,431,2,5,0,5.157773,24,3,17.958333,...,221,0.512761,113,0.262181,480,462,4,1.071926,1.113689,762.990676
1140,231,1160,235,0,2,0,4.936170,7,0,33.571429,...,100,0.425532,60,0.255319,294,290,3,1.234043,1.251064,392.991379
137,440,4414,923,62,51,0,4.782232,39,1,23.666667,...,349,0.378115,181,0.196100,750,750,5,0.812568,0.812568,1446.327517


### Features excluded from df1:

In [9]:
x = df1.drop(['essayid',
              'score',
              'punctuations',
              'avg_word_length',
              'avg_word_sentence',
              'POS/total_words',
              'prompt_words/total_words',
              'synonym_words/total_words'
             ],
             axis = 1)
y = df1['score']

In [10]:
df2 = pd.read_csv('assignment2_data/Essay-Features-Submission.csv')

In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   essayid                    199 non-null    int64  
 1   chars                      199 non-null    int64  
 2   words                      199 non-null    int64  
 3   commas                     199 non-null    int64  
 4   apostrophes                199 non-null    int64  
 5   punctuations               199 non-null    int64  
 6   avg_word_length            199 non-null    float64
 7   sentences                  199 non-null    int64  
 8   questions                  199 non-null    int64  
 9   avg_word_sentence          199 non-null    float64
 10  POS                        199 non-null    float64
 11  POS/total_words            199 non-null    float64
 12  prompt_words               199 non-null    int64  
 13  prompt_words/total_words   199 non-null    float64

### Modifications in df2

In [12]:
df2['stemmed/total_words'] = df2['stemmed']/df2['words']
df2['unstemmed/total_words'] = df2['unstemmed']/df2['words']
df2['related_words'] = df2['prompt_words'] + df2['POS'] + df2['synonym_words']

## 3. Supervised Learning

Supervised machine learning also known as supervised learning can be intepreted by the manipulation of labeled datasets to train algorithms that are able to categorise data or accurately predict results. In this df1 dataset, x is the input data and y is the labeled data. Data labeling is an important in data preprocessing for supervised machine learning, where both input and output data are labeled for classification to provide a learning basis for future data processing.

The implementation of training set on test set is crucial to validate the model built. The models generated are to predict the unknown score which is named as the test set. The sklearn.model_selection.train_test_split() is used to randomly split arrays or matrices into training and testing subsets, which are x_train, x_test, y_train, y_test. In this case, value of 0.25 is set as the test size, thus the 25% of dataset is splited as the test dataset and the remaining 75% are training dataset.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
 x, y, test_size = 0.25, random_state = 42
)

### Normalisation

When the distribution of data is unknown or the data doesn't not normally distributed, normalising data is important to leverage the numerical data without distorting the differences in the ranges of values. For instance, consider this df1 dataset containing two features, punctuations, and characters. Where punctuation ranges from 0 to 1, whereas character ranges from 236 to 4332. Thus, data normalisation allows to the variable to be at the same range.

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

### Classification

**Classification** is a supervised learning technique that categorize an unseen observation into one of the sub-groups that exist in the **training dataset**. It is supervised learning because the training data has prior knowledge of the categories as it has the target labels (**labeled dataset**). 

Difference between binary and multi-class classification: 

- **Binary classification**

   Binary classification are tasks where specimens are assigned only one of two classes (Brownlee 2021). Examples include logistic Regression, Support Vector Machines (SVM), and more are algorithms designed for binary classification problems.
   

- **multi-class classification**

    In contrary to the multi-class classification which specimens are assigned only one of more than two classes. Heuristic approaches for multi-class classification tasks including **One-vs-Rest (OvR)** and **One-vs-One (OvO)** split a multi-class classification problem into multiple classification datasets and train each binary classification model (Brownlee 2021).

## 4. Support Vector Machine 

Support vector machines (SVM) is a linear model consists a set of supervised learning techniques used for classification, regression, and outliers identification. There are specialised SVMs such as **support vector regression (SVR)** and **support vector classification (SVC)** designed to tackle particular machine learning problems (Pupale, 2018). There are two different types of SVMs, one is the simple SVM generally applied for linear regression and classification problems . 

Besides, SVMs also use **Kernel** functions to systematically find support vector classifiers in higher dimensions as it provides more flexibility for non-linear data so that more features can be added to fit a hyperplane instead of a two-dimensional space. The functions include **linear**, **polynomial**, **Gaussian Radial Basis Function (RBF)**, **Sigmoid**, and others (Awasthi 2020). 

RBF kernel SVM is applied in this report in order to target non-linear problems. Although RBF is a non-linear kernel, it utilises linear dicision region to generate non-linear combinations of the dataset's features to elevate the sample data onto a higher dimension space where linear hypersurface that partitions the underlying vector space into two sets, one for each class.  

In [15]:
clf = svm.SVC(kernel = 'rbf',
              gamma = 'auto')
clf.fit(X_train,y_train)
pred_clf = clf.predict(X_test)

### Confusion matrix  (Wisdom 2019)

This is a 6-class matrix with essay scores ranging from 1 to 6. 

Total number of true positives (classes in diagonal):
- score 1: 1
- score 2: 13
- score 3: 96
- score 4: 124
- score 5: 1
- score 6: 0
                               
Total number of false negatives (classes in corresponding rows):
- score 1: 3 + 1 = 4
- score 2: 10 + 1 = 11
- score 3: 2 + 35 = 37
- score 4: 32
- score 5: 13
- score 6: 1
                                
Total number of false positives (classes in corresponding columns):
- score 1: 0
- score 2: 3 + 2 = 5
- score 3: 1 + 10 + 32 = 43
- score 4: 1 + 35 + 13 + 1 = 50
- score 5: 1
- score 6: 0

Total number of true negatives (sum of all columns and rows of certain class excluding others class's column and row):
- score 1: 13 + 10 + 1 + 2 + 96 + 35 + 32 + 124 + 13 + 1 + 1 = 328
- score 2: 1 + 1 + 96 + 35 + 32 + 124 + 13 + 1 + 1 = 304
- score 3: 1 + 3 + 13 + 1 + 124 + 13 + 1 + 1 = 157
- score 4: 1 + 3 + 1 + 13 + 10 + 2 + 96 + 1 = 127
- score 5: 1 + 3 + 1 + 13 + 10 + 1 + 2 + 96 + 35 + 32 + 124 + 1 = 319
- score 6: 1 + 3 + 1 + 13 + 10 + 1 + 2 + 96 + 35 + 32 + 124 + 13 + 1 = 332

In [16]:
print(confusion_matrix(y_test,pred_clf))
print(classification_report(y_test,pred_clf))

[[  1   3   1   0   0   0]
 [  0  13  10   1   0   0]
 [  0   2  96  35   0   0]
 [  0   0  32 124   0   0]
 [  0   0   0  13   1   0]
 [  0   0   0   1   0   0]]
              precision    recall  f1-score   support

           1       1.00      0.20      0.33         5
           2       0.72      0.54      0.62        24
           3       0.69      0.72      0.71       133
           4       0.71      0.79      0.75       156
           5       1.00      0.07      0.13        14
           6       0.00      0.00      0.00         1

    accuracy                           0.71       333
   macro avg       0.69      0.39      0.42       333
weighted avg       0.72      0.71      0.69       333



C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision of each score are above 0.7 due to the low to no false positives found within the class. There are two classes (score 3 and 4) has sensitivity (recall) greater than 0.7. Score 5 shows the lowest recall of 0.07 because it has high number of false negative to true positive ratio (13 : 1). 

### Quadratic Weighted Kappa (QWK)

Quadratic weighted kappa (QWK) measures the agreement between two ratings. QWk typically changes from 0 to 1, which measures the agreement between raters from random to complete (Arora 2019). When the metric go below 0, there is less agreement between the raters than expected by chance. The QWK is calculated between the scores which are expected and the predicted scores. 

A perfect QWK score of 1.0 is granted when both the predictions and actuals are the same. Whereas, the least possible score is -1 which is given when the predictions are furthest away from actuals (Arora 2019). The aim is to get as close to 1 as possible. This model shows a general QWK score of 0.667 which indicates high aggreement between the predicted and test score.

In [17]:
cohen_kappa_score(y_test, pred_clf, labels=None, weights= 'quadratic')

0.667338879085791

## 5. Kaggle Dataset

In [18]:
x_test = df2.drop(['essayid',
                   'punctuations',
                   'avg_word_length',
                   'avg_word_sentence',
                   'POS/total_words',
                   'prompt_words/total_words',
                   'synonym_words/total_words'
                  ],
                  axis = 1)

In [19]:
x_df2 = sc.fit_transform(x_test)
y_pred_df2 = clf.predict(x_df2)

In [21]:
df3 = pd.read_csv('assignment2_data/test1.csv')

In [23]:
df_kaggle = df3.assign(score = y_pred_df2)
df_kaggle.to_csv("assignment2_data/test2.csv",index=False)

In [24]:
df_kaggle.head(10)

,essayid,score
0,1623,4
1,1143,3
2,660,3
3,1596,4
4,846,4
5,868,4
6,145,3
7,500,3
8,652,3
9,214,3


# Conclusion

As a summary, the trained model was tested with the generated test dataset. The outcome of result showed a standard QWK score of 0.656, with an accuracy of up to 71% on the test dataset. There are only several features selected for the training model, however some features has been excluded including 'essayid', 'punctuations', 'avg_word_length', 'avg_word_sentence', 'POS/total_words', 'prompt_words/total_words', and 'synonym_words/total_words' which are appeared to be less significant in predicting the score. Although the dataset provided is not sufficient in predicting score with high level of accuracy and precision, as type of words and vocabulary used are also important in determining the essay score. However, overall the developed model is dependable and can be upgraded by applying more features and training on different type of models.

# References

Arora, A., 2019. Quadratic Kappa Metric explained in 5 simple steps. [online] Kaggle.com. Available at: <https://www.kaggle.com/aroraaman/quadratic-kappa-metric-explained-in-5-simple-steps> [Accessed 27 April 2021].

Awasthi, S., 2020. Seven Most Popular SVM Kernels. [online] Dataaspirant. Available at: <https://dataaspirant.com/svm-kernels/> [Accessed 27 April 2021].

Brownlee, J., 2021. One-vs-Rest and One-vs-One for Multi-Class Classification. [online] Machine Learning Mastery. Available at: <https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/#:~:text=Binary%20classification%20are%20those%20tasks,Classification%20tasks%20with%20two%20classes.> [Accessed 27 April 2021].

Pupale, R., 2018. Support Vector Machines(SVM) — An Overview. [online] Towards Data Science. Available at: <https://towardsdatascience.com/https-medium-com-pupalerushikesh-svm-f4b42800e989#:~:text=SVM%20or%20Support%20Vector%20Machine,separates%20the%20data%20into%20classes.> [Accessed 27 April 2021].

Wisdom, B., 2019. Understanding the Confusion Matrix (II). [online] DEV Community. Available at: <https://dev.to/overrideveloper/understanding-the-confusion-matrix-264i#:~:text=%E2%80%8B%20Precision%20is%20a%20multi,True%20Positives%20and%20False%20Positives.> [Accessed 27 April 2021].